<a href="https://colab.research.google.com/github/heliocarrara/Python-IA/blob/main/Sistema_para_busca_de_documentos_com_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema para busca de documentos com **Gemini**

## Instalando e importando bibliotecas

In [1]:
!pip install -U -q google-generativeai

In [6]:
import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Buscando os modelos de embeddings disponíveis

In [7]:
for modelo in genai.list_models():
  if 'embedContent' in modelo.supported_generation_methods:
    print(modelo.name)

models/embedding-001
models/text-embedding-004


In [23]:
model = "models/embedding-001"

## Testando Embbeddings

### Entendendo sobre o processo de input embedContent

In [38]:
def print_result_embed(result):
  print(str(result['embedding'])[:50], '... TRIMMED ...')
  print("\n Quantidade: ", len(result['embedding']))

In [39]:
#Exemplo 1
text = "Hello World"
result = genai.embed_content(model="models/embedding-001", content=text)

print_result_embed(result)

[0.046566796, -0.037675645, -0.027483625, -0.02519 ... TRIMMED ...

 Quantidade:  768


In [40]:
#Exemplo 2
text = "Não importa o tamanho da frase"
result = genai.embed_content(model=model, content=text)

print_result_embed(result)

[0.0118317325, -0.04640216, -0.006971612, -0.01099 ... TRIMMED ...

 Quantidade:  768


In [41]:
#Exemplo com string formatada
title = "A próxima geração de IA para devs"
sample_text = (" Título: A próxima geração de IA para dev"
  "\n"
  "Artigo completo:\n"
  "\n"
  "Gemini API e o Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa"
  )
result = genai.embed_content(model=model,
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print_result_embed(result)

[0.049624845, -0.02966441, -0.005742258, 0.0226721 ... TRIMMED ...

 Quantidade:  768


In [42]:
# Lista de documentos a serem buscados
documents = [
    {
        "Titulo": "Operação do sistema de controle climático",
        "Conteúdo": "O Googlecar tem um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema, siga estas etapas:\n\n1. Pressione o botão de controle climático no console central.\n2. Use os botões para cima e para baixo para ajustar a temperatura desejada.\n3. Use o botão de ventilador para ajustar o fluxo de ar desejado.\n4. Para desligar o sistema de controle climático, pressione o botão de controle climático novamente.\n\n**Observações:**\n* O sistema de controle climático também pode ser controlado usando a tela sensível ao toque no console central.\n* Se você tiver alguma dúvida sobre como operar o sistema de controle climático, consulte o manual do proprietário do seu Googlecar."
    },
    {
        "Titulo": "Touchscreen",
        "Conteúdo": "O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação, entretenimento, comunicação e configurações. Para usar a tela sensível ao toque, siga estas etapas:\n\n1. Toque na tela para abrir um aplicativo ou função.\n2. Deslize o dedo pela tela para rolar por listas ou opções.\n3. Toque e segure um item para abri-lo em um menu pop-up.\n4. Pince ou afaste os dedos para aumentar ou diminuir o zoom.\n5. Para fechar um aplicativo ou função, toque no botão Voltar na parte inferior da tela.\n\n**Observações:**\n* A tela sensível ao toque também pode ser controlada usando comandos de voz.\n* Se você tiver alguma dúvida sobre como usar a tela sensível ao toque, consulte o manual do proprietário do seu Googlecar."
    },
    {
        "Titulo": "Mudança de marchas",
        "Conteúdo": "Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada. As posições da alavanca de câmbio são as seguintes:\n\n* **P:** Estacionamento\n* **R:** Ré\n* **N:** Neutro\n* **D:** Drive\n* **L:** Baixa marcha\n\n**Observações:**\n* Se você não tiver certeza de qual marcha usar, consulte o manual do proprietário do seu Googlecar.\n* Você também pode usar os botões no volante para trocar as marchas em alguns modelos de Googlecar."
    }
]



In [43]:
#Transformando os documentos em dataframe

df = pd.DataFrame(documents)

df.columns = ["Titulo", "Conteudo"]

df

,Titulo,Conteudo
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...


In [46]:
# Fazendo a função para definir a coluna embedding
def embed_fn(title, text, model):
  return genai.embed_content(model=model,
                             content=text,
                             title=title,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [47]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"], model), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...,"[-0.015592019, -0.025081286, -0.040180206, 0.0..."
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...,"[0.008534755, -0.03925692, 0.01427669, -0.0104..."
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...,"[-0.009098142, -0.031871554, -0.007832273, 0.0..."


In [48]:
# Função de busca por embedding

def gerar_e_buscar_consulta(consulta, base, model):
  embedding_consulta = genai.embed_content(model=model,
                             content=consulta,
                             task_type="RETRIEVAL_QUERY")['embedding']

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_consulta)

  indice = np.argmax(produtos_escalares)

  return df.iloc[indice]["Conteudo"]

### Gerando consulta via prompt e documentos

In [58]:
consulta = "Como eu troco a marcha no carro do google? To pensando em comprar um..."

melhor_documento = gerar_e_buscar_consulta(consulta, df, model)

print(melhor_documento)

Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada. As posições da alavanca de câmbio são as seguintes:

* **P:** Estacionamento
* **R:** Ré
* **N:** Neutro
* **D:** Drive
* **L:** Baixa marcha

**Observações:**
* Se você não tiver certeza de qual marcha usar, consulte o manual do proprietário do seu Googlecar.
* Você também pode usar os botões no volante para trocar as marchas em alguns modelos de Googlecar.


### Mesclando com o modelo generativo

In [65]:
generation_config = {
    "temperature" : 0,
    "candidate_count": 1
}

In [66]:
prompt = f"Imagine que você é um funcionário que trabalha no suporte de vendas de uma empresa. Fiz essa pergunta: {consulta} e os embeddings decidiram que esse é o melhor resultado. Reescreva esse texto de uma forma mais decontraída e meio séria, sem adicionar informações que não facam parte do contexto: {melhor_documento}"

model_2 = genai.GenerativeModel("gemini-1.0-pro", generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

Ei, cara! Então você tá pensando em pegar um Googlecar, hein? Legal! Sobre a troca de marchas, é moleza. O carro tem câmbio automático, então você só precisa mexer na alavanca pra escolher a marcha que quer.

Aqui estão as marchas:

* **P:** Pra estacionar
* **R:** Ré
* **N:** Neutro
* **D:** Pra dirigir
* **L:** Marcha baixa

Se ficar na dúvida, dá uma olhada no manual do carro. Alguns modelos também têm botões no volante pra trocar as marchas.
